In [1]:
import sys
!{sys.executable} -m pip install cloudscraper
!{sys.executable} -m pip install selenium

In [3]:
#get url response
import requests
from bs4 import BeautifulSoup
import urllib
import cloudscraper
import json
import time
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By

#saves reviews in dataframe
import pandas as pd

In [4]:
#a means of connecting to a chrome server used to fully open the metacritic pages.
cService = webdriver.ChromeService(executable_path=r'C:/Users/DBeye/Downloads/chromedriver-win64/chromedriver-win64/chromedriver.exe')

#used to access the metaritic page.

url = 'https://web.archive.org/web/20180620141218/http://www.metacritic.com/game/wii/sonic-and-the-secret-rings/user-reviews'

try:
  driver = webdriver.Chrome(service=cService)
  driver.get(url)

  #code to maximize window for long enough to scrape all
  # of the information.
  driver.maximize_window()
  
  
  #code to scroll down the page; scroll_num will be adjusted based on the number of reviews.
  scrollDown = 0
  #determines how far to scroll down for each page.
  scroll_num = 800
  #scrolls down the page until it reaches the end.
  
  time.sleep(20)
  while True:
    scrollDown += 1
    driver.execute_script("window.scrollBy(0,20)")
    if scrollDown > scroll_num:
      break
  """"""
  time.sleep(10)

finally:
  html = driver.page_source
  driver.quit()

In [5]:
#used to store review info.
review_dict = {}

#used to parse the html code, find all of the review
soup = BeautifulSoup(html, 'html.parser')

#print(soup.find_all('div',class_="c-siteReview g-bg-gray10 u-grid g-outer-spacing-bottom-large"))

In [ ]:
#find all li elements containing the individual reviews.
import re


#there are reviews labeled as "first-review" and "last-review"; we need to keep separate cases for them.
#this means that we need to create special indexes for them.
ult_index = 0

for index, review in enumerate(soup.find_all('li',class_="review user_review")):
  #print(soup.find_all('li',class_="review user_review"))
  review_info = {}
  #store the review grade with empty spaces removed.
  if review.find('div',class_="review_grade"):
    review_info['rating'] = review.find('div',class_="review_grade").text.replace("\n","").replace("\n","")
  #store the date that the review was published.
  if review.find('div',class_="date"):
    review_info['date'] = review.find('div',class_="date").text
  #store the user who posted the review.
  if review.find('div',class_="name"):
    review_info['user'] = review.find('div',class_="name").text.replace("\n","").replace("\n","")
  #storing empty string as review.
  review_info['review'] = ""
  
  review_dict[index] = review_info
  ult_index += 1


#there are reviews labeled as "first-review" and "last-review"; we need to use the ultimate index for them.
for index, review in enumerate(soup.find_all('li',class_="review user_review first_review")):
  review_info = {}
  #store the review grade with empty spaces removed.
  if review.find('div',class_="review_grade"):
    review_info['rating'] = review.find('div',class_="review_grade").text.replace("\n","").replace("\n","")
  #store the date that the review was published.
  if review.find('div',class_="date"):
    review_info['date'] = review.find('div',class_="date").text
  #print("FIRST REVIEW: ", review_info)
  #store the user who posted the review.
  if review.find('div',class_="name"):
    review_info['user'] = review.find('div',class_="name").text.replace("\n","").replace("\n","")
  #storing empty string as review.
  review_info['review'] = ""
  
  review_dict[ult_index] = review_info
  ult_index += 1


for index, review in enumerate(soup.find_all('li',class_="review user_review last_review")):
  review_info = {}
  #store the review grade with empty spaces removed.
  if review.find('div',class_="review_grade"):
    review_info['rating'] = review.find('div',class_="review_grade").text.replace("\n","").replace("\n","")
  #store the date that the review was published.
  if review.find('div',class_="date"):
    review_info['date'] = review.find('div',class_="date").text
  #store the user who posted the review.
  if review.find('div',class_="name"):
    review_info['user'] = review.find('div',class_="name").text.replace("\n","").replace("\n","")
  #storing empty string as review.
  review_info['review'] = ""
  
  review_dict[ult_index] = review_info
  ult_index += 1
  #print("FIRST REVIEW: ", review_info)

print(review_dict)
  


ValueError: time data 'Dec  8, 2007' does not match format '%Y-%m-%d'

In [7]:
print(json.dumps(review_dict, indent=4))

{
    "0": {
        "rating": "6",
        "date": "Dec  8, 2007",
        "user": "TommyH.",
        "review": ""
    },
    "1": {
        "rating": "1",
        "date": "Apr 28, 2008",
        "user": "BubbaS.",
        "review": ""
    },
    "2": {
        "rating": "4",
        "date": "Jun  8, 2012",
        "user": "Zilcell",
        "review": ""
    },
    "3": {
        "rating": "8",
        "date": "Nov  4, 2012",
        "user": "LaytonLink1",
        "review": ""
    },
    "4": {
        "rating": "7",
        "date": "Nov  7, 2013",
        "user": "mewkid1234",
        "review": ""
    },
    "5": {
        "rating": "7",
        "date": "Apr  3, 2015",
        "user": "sdc",
        "review": ""
    },
    "6": {
        "rating": "4",
        "date": "May 14, 2016",
        "user": "Shadow1",
        "review": ""
    },
    "7": {
        "rating": "8",
        "date": "Aug 29, 2010",
        "user": "Sonicphoto",
        "review": ""
    },
    "8": {
        "rati

In [8]:
#first, we'll calculate 
numerator = 0 
denominator = 0
above_7 = 0 
num_positive = 0 
num_mixed = 0 
num_negative = 0
total_ratings = 0
for number, info in zip(review_dict,review_dict.values()):
  numerator += int(info['rating'])
  denominator += 10
  if int(info['rating']) >= 8:
    num_positive += 1 
  if int(info['rating']) <= 7 and int(info['rating']) >= 4:
    num_mixed += 1
  if int(info['rating']) <= 3:
    num_negative += 1
  total_ratings += 1
  print(info['date'],info['rating'], num_positive, num_mixed, num_negative)

#next, we'll add our target numbers to see what we want to get.
#we want our target average rating to be hit within a 1.0 range.

Dec  8, 2007 6 0 1 0
Apr 28, 2008 1 0 1 1
Jun  8, 2012 4 0 2 1
Nov  4, 2012 8 1 2 1
Nov  7, 2013 7 1 3 1
Apr  3, 2015 7 1 4 1
May 14, 2016 4 1 5 1
Aug 29, 2010 8 2 5 1
Jan 26, 2012 4 2 6 1
Mar 13, 2012 0 2 6 2
Feb 19, 2014 5 2 7 2
Aug 22, 2014 2 2 7 3
Jun 13, 2011 5 2 8 3
Mar 15, 2007 10 3 8 3
Nov 14, 2010 8 4 8 3
Mar 26, 2013 5 4 9 3
Jun  7, 2011 4 4 10 3
Oct  6, 2013 5 4 11 3
Apr  9, 2013 1 4 11 4
May  6, 2014 1 4 11 5
Nov  6, 2013 10 5 11 5
Feb 19, 2014 5 5 12 5
Apr  8, 2014 3 5 12 6
Nov 12, 2014 1 5 12 7
Mar  8, 2008 9 6 12 7
Mar 10, 2007 8 7 12 7
Mar  5, 2007 6 7 13 7
May  4, 2007 8 8 13 7
May  9, 2007 9 9 13 7
Jun 10, 2007 9 10 13 7
Aug 30, 2007 3 10 13 8
Nov  5, 2007 5 10 14 8
Feb 24, 2007 6 10 15 8
Feb 24, 2007 9 11 15 8
Mar 15, 2007 5 11 16 8
Apr  6, 2007 7 11 17 8
Jul  2, 2007 1 11 17 9
Feb 21, 2007 10 12 17 9
Feb 22, 2007 5 12 18 9
Feb 20, 2007 10 13 18 9
Feb 27, 2007 2 13 18 10
Mar 10, 2007 5 13 19 10
Mar 31, 2007 5 13 20 10
Mar  7, 2007 6 13 21 10
Apr 23, 2007 0 13 21 11
M

In [22]:
target_average = 6.4
all_ratings = 205
target_green_num = 91 
target_yellow_num = 60 
target_red_num = 54
current_total_ratings = 0
current_numerator = 0 
current_denominator = 0
current_green_num = 0
current_yellow_num = 0
current_red_num = 0
#ADD FUNCTION TO CHECK THE REVIEW DATE.
for number, info in zip(review_dict,review_dict.values()):
  current_total_ratings += 1
  current_numerator += int(info['rating'])
  current_denominator += 10
  if int(info['rating']) >= 8:
    current_green_num += 1 
  if int(info['rating']) <= 7 and int(info['rating']) >= 4:
    current_yellow_num += 1 
  if int(info['rating']) <= 3:
    current_red_num += 1
print(all_ratings,current_total_ratings,current_green_num,current_yellow_num,current_red_num)

test_answer = float(float(all_ratings)/float(float(all_ratings)-float(current_total_ratings))) * float(float(target_average)-(float(float(current_numerator)/float(current_denominator))*10.0*float(float(current_total_ratings)/float(all_ratings))))
print(all_ratings-current_total_ratings, target_green_num-current_green_num, target_yellow_num-current_yellow_num, target_red_num-current_red_num,test_answer)

205 76 21 38 17
129 70 22 37 6.9147286821705425


In [49]:
import csv 

csv_filename = "sonic-secret-rings-user.csv"

fieldnames = ['rating','date','user','review']

with open(csv_filename,mode='w',newline='') as file:
  writer = csv.DictWriter(file,fieldnames=fieldnames)
  writer.writeheader()
  for r in review_dict:
    writer.writerow({field: review_dict[r].get(field) or r for field in fieldnames})